In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
train = pd.read_csv('./output/train.csv')    
test = pd.read_csv('./output/test.csv')    
test2 = pd.read_csv('./output/test2.csv')
y_train=pd.read_csv('./output/y_train.csv')

In [3]:
test.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,x_dir,y_dir
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,42197,42617,0.0,0.0,0.0,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07,-302.789015,-46798.074140
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,44497,44497,0.0,0.0,0.0,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07,-302.789015,-46798.074140
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,45267,47651,0.0,0.0,0.0,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07,0.000000,0.000000
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,48333,49393,0.0,0.0,0.0,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07,7997.192219,102495.208073
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,54212,54632,0.0,0.0,0.0,3.773118e+06,-1.914490e+07,NaN,NaN,-253.808439,-31208.509376


In [4]:
from sklearn.ensemble import RandomForestRegressor


x_train = train.drop(columns=['hash','trajectory_id'])
x_test = test2.drop(columns=['hash','trajectory_id'])

model = RandomForestRegressor(max_depth=128,
                              n_estimators=128,                               
                              random_state=9471,
                              n_jobs=-1,
                              verbose=True,
                             oob_score=True,
                             )
model.fit(x_train, y_train)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 128 out of 128 | elapsed:  4.6min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=128,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=-1,
           oob_score=True, random_state=9471, verbose=True,
           warm_start=False)

In [5]:
pred = model.predict(x_test)
#pred = pd.DataFrame(data=pred,columns=['x_exit','y_exit'])
pred = pd.DataFrame({'x_exit':pred[:,0],'y_exit':pred[:,1]})

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 128 out of 128 | elapsed:    0.8s finished


In [6]:
def within_measure(x, y):
    #  3750901.5068 ≤ 𝑥 ≤ 3770901.5068
    #  −19268905.6133 ≤ 𝑦 ≤ −19208905.6133
    if 3750901.5068 <= x and x <= 3770901.5068 and -19268905.6133 <= y and y <= -19208905.6133:
        return 1
    else:
        return 0

X = pred['x_exit'].values 
Y = pred['y_exit'].values

city = [within_measure(x, y) for x,y in zip(X,Y)]

In [7]:
submission= pd.DataFrame()
trajectory_id = test2['trajectory_id']
submission['id']=trajectory_id 
submission['target'] = city
submission.to_csv('./output/submission_forest.csv',index=False)    
submission[:20]

,id,target
0,traj_00032f51796fd5437b238e3a9823d13d_31_5,0
1,traj_000479418b5561ab694a2870cc04fd43_25_10,0
2,traj_000506a39775e5bca661ac80e3f466eb_29_5,1
3,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,0
4,traj_00063a4f6c12e1e4de7d876580620667_3_4,0
5,traj_0006535be25bb52dd06983447880c964_5_12,0
6,traj_0006f84bb33ec929d1cda7686f861d0a_31_3,1
7,traj_00093ae562586aed0e053b8431e8ace4_23_10,0
8,traj_000c739e444a70e1804d757a0580caaa_31_3,1
9,traj_000d479078af08618bddc7f09082b8c3_11_6,0


In [8]:
min(city)

0

In [9]:
max(city)

1